In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import nltk

In [2]:
# join all my data in one file (I have 16 csv file)
dfs = []
for i in range(1, 17):
    dfs.append(pd.read_csv(f'data/wuzzuf_job_data_20-12-2022_{i}.csv', encoding='utf-16', delimiter='~'))
df = pd.concat(dfs)

In [3]:
# save the collected data to csv file
df.to_csv('data/wuzzuf_job_data_20-12-2022.csv', sep='~', encoding='utf-16')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25161 entries, 0 to 633
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_title          25161 non-null  object
 1   job_type           25161 non-null  object
 2   job_categories     25161 non-null  object
 3   job_description    25161 non-null  object
 4   job_requirements   24968 non-null  object
 5   job_location       25161 non-null  object
 6   company_name       25161 non-null  object
 7   company_location   21434 non-null  object
 8   company_field      20968 non-null  object
 9   company_size       21434 non-null  object
 10  experience_needed  25161 non-null  object
 11  career_level       25161 non-null  object
 12  education_level    25161 non-null  object
 13  gender             25161 non-null  object
 14  salary             25161 non-null  object
 15  skills_and_tools   25161 non-null  object
 16  link               25161 non-null  object


## Explore The Data

In [5]:
df.head()

,job_title,job_type,job_categories,job_description,job_requirements,job_location,company_name,company_location,company_field,company_size,experience_needed,career_level,education_level,gender,salary,skills_and_tools,link
0,Cost Accountant,Full Time,Accounting/Finance,Preparing the cost of products and operations ...,Bachelor’s Degree in accountingProficient use ...,"Bourj Alarab, Alexandria",Confidential Company,NaN,NaN,NaN,0 to 1 year,Entry Level (Junior Level / Fresh Grad),Bachelor's Degree,both,Confidential,Accounting;Finance;Cost Accounting;Financial A...,https://wuzzuf.net/jobs/p/Ag1eTeHzp26L-Cost-Ac...
1,Costing Manager - Pastry Industry FMCG,Full Time,Accounting/Finance;Hospitality/Hotels/Food Ser...,Collecting and analyzing data about company’s ...,Bachelor degree in commerce.From 7-10 years of...,"6th of October, Giza",Confidential Company,NaN,NaN,NaN,7 to 10 years,Manager,Bachelor's Degree,Male,Confidential,Food & Beverage;Cost;FMCG;Finance;Accounting;I...,https://wuzzuf.net/jobs/p/oME6Pq8IbNE4-Costing...
2,Female Junior Accountant,Full Time,Accounting/Finance,Review of controlling & applying remittance fo...,Good understanding of accounting and financial...,"6th of October, Giza",Confidential Company,NaN,NaN,NaN,1 to 3 years,Entry Level (Junior Level / Fresh Grad),Not Specified,both,Confidential,Accounting;Finance;Financial;Financial Reporting,https://wuzzuf.net/jobs/p/TJGlRdDJ6Z8a-Female-...
3,Collection Supervisor,Full Time,Accounting/Finance,Preparing the monthly collection plan to be ap...,Pervious experience in FMCG is preferred.Has a...,"Maadi, Cairo",Abudawood Group Egypt,"Cairo, Egypt",FMCG,More than 1000 employees,2 to 6 years,Experienced (Non-Manager),Bachelor's Degree,both,Confidential,Collection;Accounting,https://wuzzuf.net/jobs/p/I3NJ1DK4FzW3-Collect...
4,Senior Costing Accountant,Full Time,Accounting/Finance,Leads and coordinates the analysis of internal...,"Bachelor degree in Accounting, finance Experie...","Cairo, Egypt",Cairo Three A Poultry,Egypt,Agriculture/Fishing/Farming,More than 1000 employees,5 to 8 years,Experienced (Non-Manager),Not Specified,both,Confidential,Costing;Finance;Financial Analysis;Financial M...,https://wuzzuf.net/jobs/p/EUswltjrqeQU-Senior-...


In [6]:
df.tail()

,job_title,job_type,job_categories,job_description,job_requirements,job_location,company_name,company_location,company_field,company_size,experience_needed,career_level,education_level,gender,salary,skills_and_tools,link
629,Tendering Supervisor - Electrical Panels,Full Time,Engineering - Mechanical/Electrical,Job Purpose :Responsible to manage Tendering E...,Bachelor's degree or Above Electrical engineer...,"6th of October, Giza",Chint Egemac,"Giza, Egypt",Manufacturing,101-500 employees,More than 8 years,Manager,Bachelor's Degree,both,Confidential,Tendering;Low Voltage;Electrical Engineering;E...,https://wuzzuf.net/jobs/p/P9JsgSASzOFG-Tenderi...
630,Supply Chain Manager,Full Time,Logistics/Supply Chain;Operations/Management,Collaborates with other departments and stakeh...,Bachelors degree in Business or related field ...,"Quweisna, Monufya",Piel color,"Monufya, Egypt",Business Services - Other,More than 1000 employees,12 to 20 years,Manager,Bachelor's Degree,both,"Confidential, Medical insurance + social insur...",Logistics;Chain Management;Supply Chain;Procur...,https://wuzzuf.net/jobs/p/R5C0RXRtmftQ-Supply-...
631,Electrical Engineer,Full Time,Engineering - Mechanical/Electrical,Responsible for ensuring that the machinery us...,At least 4 years of experience in the field of...,"6th of October, Giza",Misr October Industrial Co -MOIC Egypt,"Giza, Egypt","Engineering Services, Manufacturing, Paper and...",11-50 employees,More than 4 years,Experienced (Non-Manager),Bachelor's Degree,both,Confidential,Electrical Engineering;Computer Skills;English...,https://wuzzuf.net/jobs/p/Qr7O1jsPb81i-Electri...
632,Mechanical Engineer,Full Time,Engineering - Mechanical/Electrical,Responsible for ensuring that the machinery us...,At least 4 years of experience in the field of...,"6th of October, Giza",Misr October Industrial Co -MOIC Egypt,"Giza, Egypt","Engineering Services, Manufacturing, Paper and...",11-50 employees,More than 4 years,Experienced (Non-Manager),Bachelor's Degree,both,Confidential,Computer Skills;Engineering;English;Machines;M...,https://wuzzuf.net/jobs/p/rHyqsT2TEHmn-Mechani...
633,Backend Developer Node.js,Full Time,IT/Software Development;Engineering - Telecom/...,About NadeerNadeer Networks is IT solutions de...,Troubleshoot and debug applications. Conduct ...,"Nasr City, Cairo",Nadeer,"Riyadh, Saudi Arabia",Computer Software,11-50 employees,2 to 5 years,Experienced (Non-Manager),Not Specified,both,Confidential,Node.js;MongoDB;jQuery;JavaScript;Computer Sci...,https://wuzzuf.net/jobs/p/89B1XFeXXNNc-Backend...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25161 entries, 0 to 633
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_title          25161 non-null  object
 1   job_type           25161 non-null  object
 2   job_categories     25161 non-null  object
 3   job_description    25161 non-null  object
 4   job_requirements   24968 non-null  object
 5   job_location       25161 non-null  object
 6   company_name       25161 non-null  object
 7   company_location   21434 non-null  object
 8   company_field      20968 non-null  object
 9   company_size       21434 non-null  object
 10  experience_needed  25161 non-null  object
 11  career_level       25161 non-null  object
 12  education_level    25161 non-null  object
 13  gender             25161 non-null  object
 14  salary             25161 non-null  object
 15  skills_and_tools   25161 non-null  object
 16  link               25161 non-null  object


In [8]:
df.duplicated().sum()

0

In [9]:
df.isnull().sum()

job_title               0
job_type                0
job_categories          0
job_description         0
job_requirements      193
job_location            0
company_name            0
company_location     3727
company_field        4193
company_size         3727
experience_needed       0
career_level            0
education_level         0
gender                  0
salary                  0
skills_and_tools        0
link                    0
dtype: int64

### Column by Column Exploration

#### Job Title Column

In [10]:
df['job_title'].unique()

array(['Cost Accountant', 'Costing Manager - Pastry Industry FMCG',
       'Female Junior Accountant', ..., 'Call Center Officer',
       'Customer Support / Call Center Agent',
       'Call Center Senior Manager (Offshore Accounts)'], dtype=object)

In [11]:
df['job_title'].nunique()

4771

In [12]:
df['job_title'].isnull().sum()

0

#### Job Type Column

In [13]:
df['job_type'].unique()

array(['Full Time', 'Part Time;Freelance / Project',
       'Full Time;Work From Home', 'Full Time;Part Time',
       'Full Time;Shift Based',
       'Part Time;Freelance / Project;Work From Home', 'Internship',
       'Full Time;Freelance / Project',
       'Freelance / Project;Work From Home', 'Part Time', 'Shift Based',
       'Work From Home', 'Full Time;Part Time;Work From Home',
       'Full Time;Part Time;Shift Based', 'Freelance / Project',
       'Part Time;Work From Home',
       'Full Time;Freelance / Project;Work From Home',
       'Full Time;Part Time;Freelance / Project',
       'Full Time;Freelance / Project;Shift Based',
       'Full Time;Shift Based;Work From Home',
       'Part Time;Freelance / Project;Shift Based',
       'Full Time;Volunteering', 'Shift Based;Work From Home'],
      dtype=object)

In [14]:
df['job_type'].nunique()

23

In [15]:
df['job_type'].isnull().sum()

0

#### Job Categories Column

In [16]:
df['job_categories'].unique()

array(['Accounting/Finance',
       'Accounting/Finance;Hospitality/Hotels/Food Services;Operations/Management',
       'Accounting/Finance;Banking',
       'Accounting/Finance;Analyst/Research',
       'Accounting/Finance;Administration',
       'Accounting/Finance;Sales/Retail', 'Accounting/Finance;Quality',
       'Accounting/Finance;Administration;Banking',
       'Accounting/Finance;Banking;Operations/Management',
       'Accounting/Finance;Administration;Human Resources',
       'Accounting/Finance;Logistics/Supply Chain;Purchasing/Procurement',
       'Accounting/Finance;Training/Instructor',
       'Accounting/Finance;Customer Service/Support',
       'Accounting/Finance;Engineering - Construction/Civil/Architecture',
       'Accounting/Finance;Operations/Management;Analyst/Research',
       'Accounting/Finance;IT/Software Development;Strategy/Consulting',
       'Accounting/Finance;IT/Software Development;Operations/Management',
       'Engineering - Mechanical/Electrical',
  

In [17]:
df['job_categories'].nunique()

739

In [18]:
df['job_categories'].isnull().sum()

0

#### Job Description Column

In [19]:
df['job_description'].unique()

array(["Preparing the cost of products and operations according to accounting principles.Processing, modifying, and issuing product cost files.Preparing all inventory-related journal entries at month's end.Preparing cost of goods sold and production reports.Participating in annual physical inventory.",
       'Collecting and analyzing data about company’s operational costs.Setting standard costs of manufactured goods.Developing and analyzing cost standards.Carrying out cost-volume-profit (CVP) analysis.Performing account reconciliations.Recommending changes to the company’s processes and policies to reduce cost and maximize profit.Advising executives and upper management on appropriate costing based on financial data.\xa0',
       'Review of controlling & applying remittance for under-collected checks on the systemReview of preparing clients’ balances updating on daily basis.Review of preparing clients’ statements.Review of preparing retrieval letters.Review of monthly reconciliations 

In [20]:
df['job_description'].nunique()

6875

In [21]:
df.sort_values('job_description').head()

,job_title,job_type,job_categories,job_description,job_requirements,job_location,company_name,company_location,company_field,company_size,experience_needed,career_level,education_level,gender,salary,skills_and_tools,link
1040,Sales Engineer (Construction Chemicals) - El D...,Full Time,Engineering - Construction/Civil/Architecture;...,\r\nA leading industrial group in Egypt is see...,\r\nBachelor Degree in Engineering (Civil or A...,"Downtown, Kafr Alsheikh",NCC X-calibur,"Cairo, Egypt",Chemicals,101-500 employees,0 to 3 years,Entry Level (Junior Level / Fresh Grad),Bachelor's Degree,both,"Confidential, net salary+car allowance",Sales;Sales Skills;Indoor Sales;Engineering;Co...,https://wuzzuf.net/jobs/p/JO32u9AbMti1-Sales-E...
339,Sales Engineer (Construction Chemicals) - El D...,Full Time,Engineering - Construction/Civil/Architecture;...,\r\nA leading industrial group in Egypt is see...,\r\nBachelor Degree in Engineering (Civil or A...,"Downtown, Kafr Alsheikh",NCC X-calibur,"Cairo, Egypt",Chemicals,101-500 employees,0 to 3 years,Entry Level (Junior Level / Fresh Grad),Bachelor's Degree,both,"Confidential, net salary+car allowance",Sales;Sales Skills;Indoor Sales;Engineering;Co...,https://wuzzuf.net/jobs/p/JO32u9AbMti1-Sales-E...
1910,Sales Engineer (Construction Chemicals) - El D...,Full Time,Engineering - Construction/Civil/Architecture;...,\r\nA leading industrial group in Egypt is see...,\r\nBachelor Degree in Engineering (Civil or A...,"Downtown, Kafr Alsheikh",NCC X-calibur,"Cairo, Egypt",Chemicals,101-500 employees,0 to 3 years,Entry Level (Junior Level / Fresh Grad),Bachelor's Degree,both,"Confidential, net salary+car allowance",Sales;Sales Skills;Indoor Sales;Engineering;Co...,https://wuzzuf.net/jobs/p/JO32u9AbMti1-Sales-E...
617,Accountant,Full Time,Accounting/Finance,\r\nAccounts payable (Responsible for the proc...,Experience is a must in Bound (tourism sector).,"Downtown, Cairo",NTS Group,"Cairo, Egypt",Travel and Tourism,101-500 employees,2 to 5 years,Experienced (Non-Manager),Bachelor's Degree,both,Confidential,Accounting;Finance;Financial Management;Accoun...,https://wuzzuf.net/jobs/p/KoF69xC9H3Hy-Account...
730,Accountant,Full Time,Accounting/Finance,\r\nAccounts payable (Responsible for the proc...,Experience is a must in Bound (tourism sector).,"Downtown, Cairo",NTS Group,"Cairo, Egypt",Travel and Tourism,101-500 employees,2 to 5 years,Experienced (Non-Manager),Bachelor's Degree,both,Confidential,Accounting;Finance;Financial Management;Accoun...,https://wuzzuf.net/jobs/p/KoF69xC9H3Hy-Account...


In [22]:
df.sort_values('job_description').tail()

,job_title,job_type,job_categories,job_description,job_requirements,job_location,company_name,company_location,company_field,company_size,experience_needed,career_level,education_level,gender,salary,skills_and_tools,link
1319,Senior Payroll Specialist,Full Time,Human Resources,● Maintain Payroll system up to date and adjus...,● Bachelor's degree in accounting or any equiv...,"Maadi, Cairo",Breadfast,Egypt,"FMCG, Food and Beverage Production",501-1000 employees,3 to 5 years,Experienced (Non-Manager),Bachelor's Degree,both,Confidential,Human Resources (HR);Payroll;Accounting;excel;...,https://wuzzuf.net/jobs/p/mxrebEcsqysD-Senior-...
291,Senior Payroll Specialist,Full Time,Human Resources,● Maintain Payroll system up to date and adjus...,● Bachelor's degree in accounting or any equiv...,"Maadi, Cairo",Breadfast,Egypt,"FMCG, Food and Beverage Production",501-1000 employees,3 to 5 years,Experienced (Non-Manager),Bachelor's Degree,both,Confidential,Human Resources (HR);Payroll;Accounting;excel;...,https://wuzzuf.net/jobs/p/mxrebEcsqysD-Senior-...
1723,ERP Implementation Consultant,Full Time,Accounting/Finance;IT/Software Development;Str...,● Participate in the Analysis sessions of vari...,NaN,"Cairo, Egypt",Premier Services and Recruitment,"Cairo, Egypt",Human Resources,51-100 employees,2 to 10 years,Experienced (Non-Manager),Not Specified,both,Confidential,Accounting;Consulting;Accoumting,https://wuzzuf.net/jobs/p/VGwFx6bVMDuC-ERP-Imp...
42,ERP Implementation Consultant,Full Time,Accounting/Finance;IT/Software Development;Str...,● Participate in the Analysis sessions of vari...,NaN,"Cairo, Egypt",Premier Services and Recruitment,"Cairo, Egypt",Human Resources,51-100 employees,2 to 10 years,Experienced (Non-Manager),Not Specified,both,Confidential,Accounting;Consulting;Accoumting,https://wuzzuf.net/jobs/p/VGwFx6bVMDuC-ERP-Imp...
2625,ERP Implementation Consultant,Full Time,Accounting/Finance;IT/Software Development;Str...,● Participate in the Analysis sessions of vari...,NaN,"Cairo, Egypt",Premier Services and Recruitment,"Cairo, Egypt",Human Resources,51-100 employees,2 to 10 years,Experienced (Non-Manager),Not Specified,both,Confidential,Accounting;Consulting;Accoumting,https://wuzzuf.net/jobs/p/VGwFx6bVMDuC-ERP-Imp...


In [23]:
df['job_description'].isnull().sum()

0

#### Job Requirements Column

In [24]:
df['job_requirements'].unique()

array(['Bachelor’s Degree in accountingProficient use of Microsoft Office and its applications. (especially Excel)Ability to work under stress and meet deadlines.Attention to Detail and Deadline-Oriented.',
       'Bachelor degree in commerce.From 7-10 years of experience in Costing.CMA Holder is a Must.Experience with Accounting System, cost and inventory systems.FMCG backgroud is a Must\xa0',
       "Good understanding of accounting and financial reporting principles and practicesBachelor's degree in accountingPresentableExcellent in the English languageExcellent knowledge of MS Office and familiarity with relevant computer software (e.g Quick Books, Tally ... etc.).Females onlyOctober & El sheikh Zayed residents are most welcome to apply",
       ...,
       '\r\nHigh education graduates.\r\nExcellent verbal and written command of the English language.\r\nExcellent conversational skills through the phone in English.\r\nExperience in the same or related field is a plus.\r\nExcellent 

In [25]:
df['job_requirements'].nunique()

6920

In [26]:
df['job_requirements'].isnull().sum()

193

#### Job location Column

In [27]:
df['job_location'].unique()

array(['Bourj Alarab, Alexandria', '6th of October, Giza', 'Maadi, Cairo',
       'Cairo, Egypt', 'Nasr City, Cairo', 'Heliopolis, Cairo',
       'Agouza, Giza', 'New Cairo, Cairo',
       '10th of Ramadan City, Sharqia', 'Ain Sokhna, Suez',
       '10th of Ramadan City, Cairo', 'Giza, Giza', 'Downtown, Cairo',
       'Haram, Giza', 'Dokki, Giza', 'Badr City, Cairo',
       'Alexandria, Egypt', 'Mohandessin, Giza', 'Agami, Alexandria',
       'Faisal, Giza', 'Giza, Egypt', 'Port Said, Egypt',
       'Faqus, Sharqia', 'Banha, Qalubia', 'khanka, Qalubia',
       'Masr Al Qadima, Cairo', 'Obour City, Cairo', 'Ameria, Alexandria',
       'Mounib, Giza', 'Red Sea, Egypt', 'Garden City, Cairo',
       'Sheikh Zayed, Giza', 'Abu Rawash, Giza', 'Alsadat City, Monufya',
       'Mokattam, Cairo', 'Hadayek Alahram, Giza', 'Smart Village, Giza',
       'Shubra, Cairo', 'Zamalek, Cairo', 'Helwan, Cairo',
       'Fayoum, Egypt', 'Beni Suef, Egypt', 'Qalubia, Egypt',
       'Smouha, Alexandria', 'She

In [28]:
df['job_location'].nunique()

175

In [29]:
df['job_location'].value_counts()

Cairo, Egypt            3327
New Cairo, Cairo        2913
Maadi, Cairo            2628
Nasr City, Cairo        2179
6th of October, Giza    2072
                        ... 
Qena, Qena                 1
Basatin, Cairo             1
Bab Alshareya, Cairo       1
Ataba, Cairo               1
Schutz, Alexandria         1
Name: job_location, Length: 175, dtype: int64

In [30]:
df['job_location'].isnull().sum()

0

#### Company Name Column

In [31]:
df['company_name'].unique()

array(['Confidential Company', 'Abudawood Group Egypt',
       ' Cairo Three A  Poultry', ..., 'Rology', 'Talent Ferry',
       'WORX Outsourcing'], dtype=object)

In [32]:
df['company_name'].nunique()

1934

In [33]:
df['company_name'].value_counts()

Confidential Company                                          3727
FlairsTech                                                     196
Talaat Moustafa Group                                          135
Fawry for Banking Technology and Electronic Payments S.A.E     127
Majorel Egypt                                                  124
                                                              ... 
Panoply Nursery                                                  1
Regypt Investment                                                1
Lavita Developments                                              1
Industrial Technical Services Center (ELZahed)                   1
WORX Outsourcing                                                 1
Name: company_name, Length: 1934, dtype: int64

In [34]:
df['company_name'].isnull().sum()

0

#### Company Location Column

In [35]:
df['company_location'].unique()

array([nan, 'Cairo, Egypt ', 'Egypt ', 'Muscat, Oman ', 'Suez, Egypt ',
       'Giza, Egypt ', 'Alexandria, Egypt ', 'Monufya, Egypt ',
       'Aden, Yemen ', 'Sharqia, Egypt ', 'Saudi Arabia ',
       'United Arab Emirates ', 'London, United Kingdom ',
       'United States ', 'Gharbia, Egypt ', 'Assiut, Egypt ', 'Japan ',
       'Port Said, Egypt ', 'Amman, Jordan ', 'North Sinai, Egypt ',
       'Riyadh, Saudi Arabia ', 'Dakahlia, Egypt ', 'Poland ',
       'Matruh, Egypt ', 'Qalubia, Egypt ', 'Dallas, United States ',
       'Nicaragua ', 'Nottingham, United Kingdom ', 'United Kingdom ',
       'Jeddah, Saudi Arabia ', 'Dubai, United Arab Emirates ',
       'Vienna, Austria ', 'Toronto, Canada ',
       'Abu Dhabi, United Arab Emirates ',
       'New York City, United States ', 'Taipei, Taiwan ',
       'Kafr Alsheikh, Egypt ', 'Canada ', 'San Diego, United States ',
       'Dammam, Saudi Arabia ', 'Pakistan ',
       'Al Ain, United Arab Emirates ', 'Zurich, Switzerland ',
       

In [36]:
df['company_location'].nunique()

59

In [37]:
df['company_location'].value_counts()

Cairo, Egypt                        12828
Giza, Egypt                          4180
Egypt                                2271
Alexandria, Egypt                     895
Sharqia, Egypt                        285
Jeddah, Saudi Arabia                  130
Dakahlia, Egypt                        72
Riyadh, Saudi Arabia                   68
Dubai, United Arab Emirates            60
United Kingdom                         46
United Arab Emirates                   45
Monufya, Egypt                         41
Poland                                 40
Qalubia, Egypt                         38
Saudi Arabia                           35
Gharbia, Egypt                         29
Aden, Yemen                            27
Port Said, Egypt                       27
Dallas, United States                  25
Bern, Switzerland                      25
San Diego, United States               25
Red Sea, Egypt                         22
Antwerpen, Belgium                     21
Suez, Egypt                       

In [38]:
df['company_location'].isnull().sum()

3727

#### Company Field Column

In [39]:
df['company_field'].unique()

array([nan, 'FMCG', 'Agriculture/Fishing/Farming', 'Cosmetics',
       'Publishing and Printing', 'Accounting and Auditing Services',
       'Information Technology Services', 'Manufacturing',
       'Food and Beverage Production', 'Consumer Electronics',
       'Real Estate/Property Management',
       'Construction - Residential & Commercial/Office, Real Estate/Property Management',
       'Construction - Residential & Commercial/Office',
       'Accounting and Auditing Services, Marketing and Advertising, Import and Export',
       'Banking, Financial Services, Non-Profit Organizations',
       'Construction - Industrial Facilities and Infrastructure',
       'Business Services - Other', 'Outsourcing/Offshoring',
       'Computer Games, Entertainment',
       'Business Services - Other, Engineering Services, Management Consulting',
       'Marketing and Advertising, Computer Software, Information Technology Services',
       'Import and Export',
       'Computer and Network Security

In [40]:
df['company_field'].nunique()

579

In [41]:
df['company_field'].value_counts()

Information Technology Services                                                     1289
Computer Software                                                                   1104
Manufacturing                                                                        865
Real Estate/Property Management                                                      815
Engineering Services                                                                 650
                                                                                    ... 
Renewables and Environment                                                             1
Insurance, Real Estate/Property Management, Investment Banking                         1
Construction - Industrial Facilities and Infrastructure, FMCG, Import and Export       1
Marketing and Advertising, Cosmetics, Personal Care and Services                       1
Computer Software, Healthcare and Medical Services, Research                           1
Name: company_field, 

In [42]:
df['company_field'].isnull().sum()

4193

#### Company Size Column

In [43]:
df['company_size'].unique()

array([nan, ' More than 1000 employees', ' 101-500 employees',
       ' 11-50 employees', ' 51-100 employees', ' 501-1000 employees',
       ' 1-10 employees', ' '], dtype=object)

In [44]:
df['company_size'].nunique()

7

In [45]:
df['company_size'].isnull().sum()

3727

#### Experience Needed Column

In [46]:
df['experience_needed'].unique()

array(['0 to 1 year', '7 to 10 years', '1 to 3 years', '2 to 6 years',
       '5 to 8 years', '4 to 7 years', '5 to 7 years',
       'More than 3 years', '10 to 15 years', '4 to 9 years',
       '2 to 4 years', '6 to 7 years', '5 to 10 years', '3 to 6 years',
       '2 to 5 years', '3 to 7 years', '0 to 3 years', '3 to 5 years',
       '3 to 10 years', '1 to 2 years', '2 to 3 years', '2 to 10 years',
       '12 to 20 years', '3 to 8 years', 'More than 15 years',
       'More than 10 years', '1 year', '6 to 8 years', '4 to 5 years',
       'More than 1 year', '5 to 6 years', '3 to 4 years',
       '10 to 12 years', '0 to 2 years', '8 to 10 years',
       'More than 5 years', '10 to 17 years', 'More than 2 years',
       '1 to 10 years', '5 to 9 years', '5 to 15 years', '4 to 6 years',
       '4 to 10 years', 'More than 20 years', '7 to 15 years',
       '1 to 7 years', '18 to 20 years', '2 to 7 years',
       'More than 7 years', 'More than 6 years', '7 to 9 years',
       '6 to 9 years

In [47]:
df['experience_needed'].nunique()

136

In [48]:
df['experience_needed'].value_counts()

3 to 5 years         2587
1 to 3 years         2248
2 to 4 years         1443
More than 3 years    1094
2 to 5 years         1080
                     ... 
10 to 11 years          2
1 to 15 years           1
9 to 20 years           1
7 to 13 years           1
0 to 8 years            1
Name: experience_needed, Length: 136, dtype: int64

In [49]:
df['experience_needed'][df['experience_needed'] == 'Not Specified'].count()

306

In [50]:
df['experience_needed'].isnull().sum()

0

#### Career Level Column

In [51]:
df['career_level'].unique()

array(['Entry Level (Junior Level / Fresh Grad)', 'Manager ',
       'Experienced (Non-Manager)',
       'Senior Management (CEO, GM, Director, Head)',
       'Student (Undergrad / Postgrad)'], dtype=object)

In [52]:
df['career_level'].nunique()

5

In [53]:
df['career_level'].isnull().sum()

0

#### Education Level Column

In [54]:
df['education_level'].unique()

array(["Bachelor's Degree", 'Not Specified', 'Diploma', 'MBA',
       "Master's Degree", 'High School (or equivalent)'], dtype=object)

In [55]:
df['education_level'].nunique()

6

In [56]:
df['education_level'].isnull().sum()

0

#### Gender Column

In [57]:
df['gender'].unique()

array(['both', 'Male', 'Female', 'Males preferred', 'Females preferred'],
      dtype=object)

In [58]:
df['gender'].nunique()

5

In [59]:
df['gender'].value_counts()

both                 22688
Male                  1511
Female                 648
Males preferred        241
Females preferred       73
Name: gender, dtype: int64

In [60]:
df['gender'].isnull().sum()

0

#### Salary Column

In [61]:
df['salary'].unique()

array(['Confidential', '3500 to 3500 EGP Per Month',
       'Confidential, Bounces', ..., '60 to 80 EGP Per Hour',
       '4000 to 5800 EGP Per Month',
       'Confidential, Medical Insurance\r\nSocial Insurance\r\nTransportation Allowance'],
      dtype=object)

In [62]:
df['salary'].nunique()

1168

In [63]:
df['salary'].value_counts()

Confidential                                                                                    18125
Confidential, Commission                                                                          267
Confidential, Bonus                                                                               212
paid                                                                                              157
Confidential, Commissions                                                                          76
                                                                                                ...  
Confidential, Social Insurance , Medical Insurance ,Transportation                                  1
Confidential, Commission + Social & Medical insurance                                               1
4000 to 8000 EGP Per Month, Commission for achieving targets\r\nImmediate bonuses and spiffs        1
Confidential, Incentives, Social Insurance, Medical Insurance, and Transportation.

In [64]:
df['salary'].isnull().sum()

0

#### Skills And Tools Column

In [65]:
df['skills_and_tools'].unique()

array(['Accounting;Finance;Cost Accounting;Financial Analysis',
       'Food & Beverage;Cost;FMCG;Finance;Accounting;Inventory;Management;Cost Control;Accounting System;Costing;Financial Analysis;Financial Management;Commerce;CMA;Pastry;bakery;patissier;Hotel, Restaurant, Cafe;horeca;Cafe;pizza',
       'Accounting;Finance;Financial;Financial Reporting', ...,
       'Customer Service;Customer Care;Customer Support;Call Center;Contact Center;English',
       'realestate;Operations Management', 'Customer Care;English'],
      dtype=object)

In [66]:
df['skills_and_tools'].nunique()

7023

In [67]:
df['skills_and_tools'].isnull().sum()

0

#### Link Column

In [68]:
df['link'].unique()

array(['https://wuzzuf.net/jobs/p/Ag1eTeHzp26L-Cost-Accountant-Alexandria-Egypt?o=1&l=bp&t=bj&bpv=np&a=Accounting-Finance-Jobs-in-Egypt',
       'https://wuzzuf.net/jobs/p/oME6Pq8IbNE4-Costing-Manager---Pastry-Industry-FMCG-Giza-Egypt?o=2&l=bp&t=bj&bpv=np&a=Accounting-Finance-Jobs-in-Egypt',
       'https://wuzzuf.net/jobs/p/TJGlRdDJ6Z8a-Female-Junior-Accountant-Giza-Egypt?o=3&l=bp&t=bj&bpv=np&a=Accounting-Finance-Jobs-in-Egypt',
       ...,
       'https://wuzzuf.net/jobs/p/Qr7O1jsPb81i-Electrical-Engineer-Misr-October-Industrial-Co--MOIC-Egypt-Giza-Egypt?o=2498&l=bp&t=bj&bpv=np&a=Engineering-Jobs-in-Egypt',
       'https://wuzzuf.net/jobs/p/rHyqsT2TEHmn-Mechanical-Engineer-Misr-October-Industrial-Co--MOIC-Egypt-Giza-Egypt?o=2499&l=bp&t=bj&bpv=np&a=Engineering-Jobs-in-Egypt',
       'https://wuzzuf.net/jobs/p/89B1XFeXXNNc-Backend-Developer-Nodejs-Nadeer-Cairo-Egypt?o=2500&l=bp&t=bj&bpv=np&a=Engineering-Jobs-in-Egypt'],
      dtype=object)

In [69]:
df['link'].nunique()

25161

In [70]:
df['link'].isnull().sum()

0

### Data Explore Conclusion
- The data may appear to be complete, but I have handled most of it during scraping by adding neutral entries such as 'both' for gender, 'confidential' for company names, and making other modifications.
- There are a lot of duplicated jobs, and despite the duplicated function displayed as 0, more investigation is needed.
- The job type column needs to be organized into main types only.
- We could try to narrow the variation in the job titles column.
- We need to extract the main category for each job from the job categories column.
- We can try to extract useful information from the job description and job requirements columns.
- We can get the job area and city from the job location.
- We can fill in the company location from the job location.
- The company field can be refined and filled in with an approximation of the job field.
- The company size can be filled in with the mode or any other method.
- The career level needs to be more explanatory (with more categories).
- The experience needed needs to be more refined with fewer categories (and be a minimum and maximum range).
- The education level need more investigation.
- The gender column need more investigation.
- The salary column needs to be refined and made into a minimum and maximum range (if applicable).
- Most of the salary column is not numeric, but I can get information such as insurance and commission jobs.

## Data Cleaning

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25161 entries, 0 to 633
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_title          25161 non-null  object
 1   job_type           25161 non-null  object
 2   job_categories     25161 non-null  object
 3   job_description    25161 non-null  object
 4   job_requirements   24968 non-null  object
 5   job_location       25161 non-null  object
 6   company_name       25161 non-null  object
 7   company_location   21434 non-null  object
 8   company_field      20968 non-null  object
 9   company_size       21434 non-null  object
 10  experience_needed  25161 non-null  object
 11  career_level       25161 non-null  object
 12  education_level    25161 non-null  object
 13  gender             25161 non-null  object
 14  salary             25161 non-null  object
 15  skills_and_tools   25161 non-null  object
 16  link               25161 non-null  object


### Remove duplicate data
To remove the duplicate data we need to find what is making the data read as unique in the duplicated function and after investigation, I have found that the same job could have more than one job URL because the companies post the same job over and over. So we need to drop link to remove duplicates

In [72]:
df.drop('link', inplace=True, axis=1)

In [73]:
df.duplicated().sum() # number of duplicated after removeing link column

17932

In [74]:
df.drop_duplicates(inplace=True) # remove duplicates

### Cleaninig Job type column
In this column I will consider the main type of the job:

[1] Full Time

[2] Part Time

[3] Freelance

[4] Internship

In [75]:
df['job_type'].value_counts()

Full Time                                       6624
Full Time;Work From Home                         202
Internship                                        86
Full Time;Shift Based                             64
Full Time;Part Time                               57
Part Time                                         49
Full Time;Freelance / Project                     20
Freelance / Project                               16
Full Time;Part Time;Freelance / Project           15
Full Time;Part Time;Work From Home                14
Full Time;Shift Based;Work From Home              14
Part Time;Freelance / Project;Work From Home      11
Work From Home                                    10
Part Time;Freelance / Project                      8
Part Time;Work From Home                           8
Full Time;Freelance / Project;Work From Home       8
Shift Based                                        6
Freelance / Project;Work From Home                 6
Full Time;Part Time;Shift Based               

In [76]:
to_be_replaced = {
    'Full Time;Work From Home': 'Full Time',
    'Full Time;Shift Based;Work From Home':'Full Time',
    'Full Time;Shift Based': 'Full Time',
    'Full Time;Part Time': 'Full Time',
    'Full Time;Freelance / Project': 'Full Time',
    'Full Time;Part Time;Freelance / Project': 'Full Time',
    'Full Time;Part Time;Work From Home': 'Full Time',
    'Part Time;Freelance / Project;Work From Home': 'Part Time',
    'Work From Home': 'Part Time',
    'Part Time;Freelance / Project': 'Part Time',
    'Part Time;Work From Home': 'Part Time',
    'Full Time;Freelance / Project;Work From Home': 'Full Time',
    'Shift Based': 'Full Time',
    'Freelance / Project;Work From Home': 'Freelance / Project',
    'Full Time;Part Time;Shift Based': 'Full Time',
    'Part Time;Freelance / Project;Shift Based': 'Part Time',
    'Full Time;Freelance / Project;Shift Based': 'Full Time',
    'Full Time;Volunteering': 'Full Time',
    'Shift Based;Work From Home': 'Full Time'
}

df['job_type'].replace(to_be_replaced, inplace=True)

In [77]:
df['job_type'].value_counts()

Full Time              7031
Part Time                90
Internship               86
Freelance / Project      22
Name: job_type, dtype: int64

### Cleaninig Job Category Cloumn
In this column I have scrapped all the job categories but there were always a main job category per job that I will extract now from the data

In [78]:
def get_the_main_job_category(job_categories):
    return job_categories.split(';')[0]

df['job_categories'] = df['job_categories'].apply(get_the_main_job_category)

In [79]:
df.isnull().sum()

job_title               0
job_type                0
job_categories          0
job_description         0
job_requirements       91
job_location            0
company_name            0
company_location     1074
company_field        1220
company_size         1074
experience_needed       0
career_level            0
education_level         0
gender                  0
salary                  0
skills_and_tools        0
dtype: int64

### Clean company Location
we can fill empty company location from job location

In [80]:
def get_company_location(row):
    if row['company_location'] is np.nan:
        return row['job_location']
    return row['company_location']

df['company_location'] = df.apply(get_company_location, axis=1)

### Clean company field
we can infer company field from job category

In [81]:
def get_company_field(row):
    if row['company_field'] is np.nan:
        try:
            return df[(df['job_categories'].str.contains(row['job_categories'])) & (df['company_field'].isnull() == False)]['company_field'].mode()[0]
        except Exception:
            return np.nan
    return row['company_field']

df['company_field'] = df.apply(get_company_field, axis=1)

Fill the rest with the mode

In [82]:
df['company_field'].fillna(df['company_field'].mode()[0], inplace=True)

### Clean Company Size

In [83]:
df['company_size'].fillna(df['company_size'].mode()[0], inplace=True)

### Clean experience needed
I will put the experience in number of categories

    [1] Min: 0 Max: 1

    [2] Min: 1 Max: 2
    
    [3] Min: 3 Max: 5
    
    [4] Min: 5 Max: 7
    
    [5] Min: 7 Max: 10
    
    [6] Min: 10 Max: 15
    
    [7] Min: 15 Max: 20
    
    [8] Min: 20 Max: 25
    
    [9] Min: 25

In [84]:
df['experience_needed'].value_counts()

3 to 5 years          738
1 to 3 years          679
2 to 4 years          416
1 to 2 years          338
2 to 5 years          317
                     ... 
4 to 15 years           1
12 to 18 years          1
More than 11 years      1
5 to 18 years           1
0 to 8 years            1
Name: experience_needed, Length: 136, dtype: int64

In [85]:
def get_experience_needed(experience):
    numbers = re.findall(r'\d+', experience)
    try:
        if int(numbers[0]) == 0:
            return 'Min: 0 Max: 1'
        elif int(numbers[0]) <= 2:
            return 'Min: 1 Max: 2'
        elif int(numbers[0]) <= 5:
            return 'Min: 3 Max: 5'
        elif int(numbers[0]) <= 10:
            return 'Min: 7 Max: 10'
        elif int(numbers[0]) <= 15:
            return 'Min: 10 Max: 15'
        elif int(numbers[0]) <= 20:
            return 'Min: 15 Max: 20'
        elif int(numbers[0]) <= 25:
            return 'Min: 20 Max: 25'
        else:
            return 'Min: 25'
    except IndexError:
        return np.nan
    
df['experience_needed'] = df['experience_needed'].apply(get_experience_needed)

In [86]:
df.isnull().sum()

job_title              0
job_type               0
job_categories         0
job_description        0
job_requirements      91
job_location           0
company_name           0
company_location       0
company_field          0
company_size           0
experience_needed    121
career_level           0
education_level        0
gender                 0
salary                 0
skills_and_tools       0
dtype: int64

Any job with Internship Job type must be in 'Min: 0 Max: 1' level

In [87]:
def internship_experience_needed(row):
    if row['experience_needed'] is np.nan and row['job_type'] == 'Internship':
        return 'Min: 0 Max: 1'
    return row['experience_needed']

df['experience_needed'] = df.apply(internship_experience_needed, axis=1)

Fill the experience needed based on the career level

In [88]:
def entry_level_experience_needed(row):
    if row['experience_needed'] is np.nan and row['career_level'] == 'Entry Level (Junior Level / Fresh Grad)':
        return df[df['career_level'].str.contains('Entry Level')]['experience_needed'].mode()[0]
    elif row['experience_needed'] is np.nan and row['career_level'] == 'Experienced (Non-Manager)':
        return df[df['career_level'].str.contains('Experienced')]['experience_needed'].mode()[0]
    elif row['experience_needed'] is np.nan and row['career_level'] == 'Manager':
        print(row['career_level'])
        return df[df['career_level'].str.contains('Manager')]['experience_needed'].mode()[0]
    return row['experience_needed']

df['experience_needed'] = df.apply(entry_level_experience_needed, axis=1)

Fill the rest with the mode

In [89]:
df['experience_needed'].fillna(df['experience_needed'].mode()[0], inplace=True)

### clean job requirments
Fill the empty job requirment with the job description

In [90]:
df['job_requirements'] = df.apply(lambda row: row['job_description'] if row['job_requirements'] is np.nan else row['job_requirements'], axis=1)

### Clean Salary column
most of salary data is not useful so I will drop the column

In [91]:
df.drop('salary', axis=1, inplace=True)

### Clean skills and tools column
most of skills_and_tools data is not useful so I will drop the column

In [92]:
df.drop('skills_and_tools', axis=1, inplace=True)

### Clean gender column

In [93]:
data_to_be_replaced = {
    'Males preferred': 'Male',
    'Females preferred': 'Female'
}

df['gender'].replace(data_to_be_replaced, inplace=True)

Make sure that both genders are accepted in the job

In [94]:
def extract_gender(row):
    if row['gender'] == 'both': 
        # In the next step and based on some observation I will try to remove non experiance numbers as I can from the text
        text = row['job_description'].lower()

        # Split the text into sentences
        sentences = nltk.sent_tokenize(text)

        for sentence in sentences:
            if re.search(r'male', sentence) and re.search(r'female', sentence):
                return 'Both'
            elif re.search(r'male', sentence):
                return 'Male'
            elif re.search(r'female', sentence):
                return 'Female'
        return 'Both'
    return row['gender']

df['gender'] = df.apply(extract_gender, axis=1)

In [95]:
df['education_level'].value_counts()

Bachelor's Degree              6570
Not Specified                   606
Master's Degree                  25
High School (or equivalent)      15
MBA                              11
Diploma                           2
Name: education_level, dtype: int64

### Clean education level column
make sure that I get most of the data right by extracting info from description and requirements columns

In [96]:
def extract_education_level(row):
    if row['education_level'] == 'Not Specified':
        text_description = row['job_description'].lower()
        text_requirements = row['job_requirements'].lower()
        
        
        # Split the text into sentences
        sentences_description = nltk.sent_tokenize(text_description)
        sentences_requirements = nltk.sent_tokenize(text_requirements)
        
        for sentence in sentences_description:
            if re.search(r'(bachelor|بكالوريوس|بكالريوس|bs degree|graduate|b.sc|bsc|مؤهلات عليا|university|مؤهل عالي)', sentence):
                return "Bachelor's Degree"
            elif re.search(r'(high school|diploma)', sentence):
                return 'High School (or equivalent)'
            elif re.search(r'master|mba', sentence):
                return "Master's Degree"
            
        for sentence in sentences_requirements:
            if re.search(r'(bachelor|بكالوريوس|بكالريوس|bs degree|graduate|b.sc|bsc|مؤهلات عليا|university|مؤهل عالي)', sentence):
                return "Bachelor's Degree"
            elif re.search(r'(high school|diploma)', sentence):
                return 'High School (or equivalent)'
            elif re.search(r'master|mba', sentence):
                return "Master's Degree"
        return np.nan
    elif row['education_level'] == 'MBA':
        return "Master's Degree"
    
    elif row['education_level'] == 'Diploma':
        return "High School (or equivalent)"
    
    return row['education_level']
    
    
df['education_level'] = df.apply(extract_education_level, axis=1)

Fill the rest with the mode

In [97]:
df['education_level'].fillna(df['education_level'].mode()[0], inplace=True)

In [98]:
df['education_level'].value_counts()

Bachelor's Degree              7150
Master's Degree                  56
High School (or equivalent)      23
Name: education_level, dtype: int64

## Save after cleaning

In [99]:
df.to_csv('cleaned_wuzzuf_data.csv', encoding='utf-16')